In [8]:
# pip install schedule

In [9]:
import pandas as pd
import numpy as np
import pickle
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import logging
import requests
import schedule
import time

In [5]:
logging.basicConfig(filename='churn_email.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [10]:
url = "https://raw.githubusercontent.com/RicardoMuchacho/bank-churn-prediction-model/master/xgb_model.pkl"

response = requests.get(url)
if response.status_code == 200:
    with open("xgb_model.pkl", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully!")
else:
    print("Failed to download file")

with open("xgb_model.pkl", "rb") as f:
    model = pickle.load(f)

File downloaded successfully!


In [11]:
customer_data = pd.read_csv('current_customers.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'current_customers.csv'

In [ ]:
customer_ids = customer_data['customer_id']
emails = customer_data['email']

features = customer_data.drop(columns=['customer_id', 'email'])

churn_probabilities = model.predict_proba(features)[:, 1]

In [ ]:
threshold1 = 0.5
threshold2 = 0.65
threshold3 = 0.8

low_risk_indices = np.where(churn_probabilities > threshold1)[0]
medium_risk_indices = np.where(churn_probabilities > threshold2)[0]
high_risk_indices = np.where(churn_probabilities > threshold3)[0]

low_risk_customers = customer_data.iloc[low_risk_indices]
medium_risk_customers = customer_data.iloc[medium_risk_indices]
high_risk_customers = customer_data.iloc[high_risk_indices]

In [ ]:
EMAIL_ADDRESS = 'your_email@banco.es'
EMAIL_PASSWORD = 'contraseña'
SMTP_SERVER = 'smtp.banco.es'
SMTP_PORT = 587

In [ ]:
def send_email_low(to_address, customer_name):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_address
        msg['Subject'] = 'Un correo para agradecerte por estar con nosotros'

        body = f"""
        Querido {customer_name},

        Sabemos que has estado con nosotros durante algún tiempo y queremos recompensar tu fidelidad y mostrarte nuestro agradecimiento.
        Ahora tienes disponibles algunas ofertas especiales para ti y tus seres queridos en préstamos familiares, de automóvil o vivienda.
        ¡No te los pierdas!

        Esperamos saber de ti pronto y poder hablar sobre nuestro futuro y éxitos en común.

        No dudes en ponerte en contacto con nuestros servicios, ¡te guiaremos a través de todos tus beneficios!

        Saludos cordiales,
        Tu servicio de atención al cliente
        """
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)

        logging.info(f"Email mandado a {to_address}")
    except Exception as e:
        logging.error(f"Failó mandar el email a {to_address}: {e}")

In [ ]:
for index, row in low_risk_customers.iterrows():
    send_email_low(row['email'], row['customer_name'])

In [ ]:
def send_email_medium(to_address, customer_name):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_address
        msg['Subject'] = 'Un correo para agradecerte por estar con nosotros'

        body = f"""
        Querido {customer_name},

        Sabemos que has estado con nosotros durante algún tiempo y queremos recompensar tu fidelidad y mostrarte nuestro agradecimiento.
        Ahora tienes disponibles algunas ofertas especiales para ti y tus seres queridos en planes de pensiones y seguros de salud.
        ¡No te los pierdas!

        Esperamos saber de ti pronto y poder hablar sobre nuestro futuro y éxitos en común.

        No dudes en ponerte en contacto con nuestros servicios, ¡te guiaremos a través de todos tus beneficios!

        Saludos cordiales,
        Tu servicio de atención al cliente
        """
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)

        logging.info(f"Email mandado a {to_address}")
    except Exception as e:
        logging.error(f"Failó mandar el email a {to_address}: {e}")

In [ ]:
for index, row in medium_risk_customers.iterrows():
    send_email_medium(row['email'], row['customer_name'])

In [ ]:
def send_email_high(to_address, customer_name):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_address
        msg['Subject'] = 'Un correo para agradecerte por estar con nosotros'

        body = f"""
        Querido {customer_name},

        Sabemos que has estado con nosotros durante algún tiempo y queremos recompensar tu fidelidad y mostrarte nuestro agradecimiento.
        Ahora tienes disponibles algunas ofertas especiales para ti y tus seres queridos en préstamos familiares, de automóvil o vivienda,
        así como descuentos muy interesantes en planes de pensiones y seguros de salud. ¡No te los pierdas!

        Esperamos saber de ti pronto y poder hablar sobre nuestro futuro y éxitos en común.

        No dudes en ponerte en contacto con nuestros servicios, ¡te guiaremos a través de todos tus beneficios!

        Saludos cordiales,
        Tu servicio de atención al cliente
        """
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)

        logging.info(f"Email mandado a {to_address}")
    except Exception as e:
        logging.error(f"Failó mandar el email a {to_address}: {e}")

In [ ]:
for index, row in high_risk_customers.iterrows():
    send_email_high(row['email'], row['customer_name'])

In [13]:
def run_script():
    print("Running churn email script")

schedule.every().monday.at("08:00").do(run_script)

while True:
    schedule.run_pending()
    time.sleep(60)

KeyboardInterrupt: 